# Multi Author Writing Style Analysis
by: Noah Syrkis


As a writer, I have always been interested in the idea of a "writing style".
I have always wondered if there was a way to quantify the style of a writer, and if so, how would one go about doing so?
This project is an attempt to build a model that does precisely that.

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
import random
from tqdm import tqdm
from functools import partial
from src.utils import get_data

## data

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
data = get_data(2, model.encode)

100%|██████████| 900/900 [00:09<00:00, 95.80it/s] 


In [10]:
def paired_samples(data_split):
    """turns data set into pair of consectuve sentences (flattens multi paragraph samples into pairs)"""
    pairs = []
    for problem_id in tqdm(data_split.keys()):
        texts = data_split[problem_id]['text']
        targets = data_split[problem_id]['truth']['changes']
        if len(texts) - 1 != len(targets):
            # TODO: fix. a few of the samples have more than one paragraph, making .readlines() wrong
            print(f'problem {problem_id} has {len(texts)} texts and {len(targets)} targets')
            continue
        for target, text1, text2 in zip(targets, texts[:-1], texts[1:]):
            pairs.append((text1, text2, target))
    random.shuffle(pairs)
    return pairs

train_pairs = paired_samples(data['train'])
valid_pairs = paired_samples(data['validation'])

100%|██████████| 900/900 [00:00<00:00, 145669.28it/s]

problem 71 has 31 texts and 25 targets
problem 260 has 6 texts and 4 targets


## models

In [ ]:
i

2